In [ ]:
from PIL import Image
from transformers import AutoProcessor, Kosmos2ForConditionalGeneration

In [ ]:
model = Kosmos2ForConditionalGeneration.from_pretrained("microsoft/kosmos-2-patch14-224").to("cuda")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")

In [ ]:
image = Image.open("../1_media/input_images/cat.jpg")
prompt = "<grounding>Describe this images:"

inputs = processor(text=prompt, images=image, return_tensors="pt")
inputs.to("cuda")
print(inputs.keys())
print("Image embeds:", inputs["image_embeds_position_mask"])

generated_ids = model.generate(
    pixel_values=inputs["pixel_values"],
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    image_embeds=None,
    image_embeds_position_mask=inputs["image_embeds_position_mask"],
    use_cache=True,
    max_new_tokens=64,
)
print(generated_ids.shape)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
processed_text = processor.post_process_generation(generated_text, cleanup_and_extract=False)
print(processed_text)

caption, entities = processor.post_process_generation(generated_text)
print(caption)

In [ ]:
caption